In [1]:
load_model = 'efficientnet_b7_based_model2.pt'

data_dir = '../../../stanford_dogs_new/'

In [2]:
# import numpy as np

from IPython.display import Image

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision.models as models
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

from torchvision.datasets import ImageFolder
from PIL import Image

from torch.autograd import Variable

from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy


from IPython.display import Image
from IPython.display import Markdown

from PIL import Image
import PIL.Image

import pandas as pd

# sklearn metrics
from sklearn import metrics
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import f1_score


## load the efficientnet_b7 based model

In [3]:
efficientnet_b7_loaded = models.efficientnet_b7(pretrained=True)

for param in efficientnet_b7_loaded.parameters():
    param.requires_grad = True

efficientnet_b7_loaded.fc = nn.Linear(in_features=2560, out_features=120, bias=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
efficientnet_b7_loaded = efficientnet_b7_loaded.to(device)



efficientnet_b7_loaded.load_state_dict(torch.load('efficientnet_b7_based_model2.pt'))
efficientnet_b7_loaded.eval()

EfficientNet(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
            (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): ConvNormActivatio

In [4]:
model = efficientnet_b7_loaded

In [5]:
# preprocess the image for the model
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
    )
])

In [8]:
path = '../../../stanford_dogs_new/test'

list_subfolders_with_paths = [f.path for f in os.scandir(path) if f.is_dir()]

# gets the breed name from subfolders name like 'n02099429-curly-coated_retriever'
def rename(name):
    return ' '.join(' '.join(name.split('-')[1:]).split('_'))

per_class_accuracy = dict()
total_accuracy = 0

y_true = []
y_pred = []

for breed_dir in list_subfolders_with_paths:
    
    breed = rename(breed_dir.split('/')[-1])
    
    images = os.listdir(breed_dir)
    
    matched = False
    matches = 0
    total_images = len(images)

    for image in images:

        input_image = Image.open(breed_dir + '/' + image)


        input_tensor = preprocess(input_image)
        if torch.cuda.is_available():
            input_tensor = Variable(input_tensor.cuda())

        input_batch = input_tensor.unsqueeze(0)
        out = model(input_batch)

        probabilities = torch.nn.functional.softmax(out[0], dim=0)
        # print(probabilities)

        with open("../../stanford_dogs_breeds_classes_final.txt", "r") as f:
            categories = [s.strip() for s in f.readlines()]

        predicted_breeds = []
        top1_prob, top1_catid = torch.topk(probabilities, 1)
        for i in range(top1_prob.size(0)):
            # predicted_breeds.append([categories[top3_catid[i]], top3_prob[i].item()*100])
            if(top1_catid[i] <= 120):
                predicted_breed = categories[top1_catid[i]]
                y_true.append(breed)
                y_pred.append(predicted_breed)

        # list to be used directly by the application (predicted_breed, probability)
        # print("predicted_breeds are: \n", predicted_breed)
        if breed == predicted_breed:
            matched = True
            matches = matches + 1

        per_class_accuracy[breed] = matches / total_images * 100

In [9]:
df = pd.DataFrame(list(zip(y_true, y_pred)),
               columns =['Actual_breed', 'Predicted_breed'])

df

,Actual_breed,Predicted_breed
0,Pomeranian,Pomeranian
1,Pomeranian,Pomeranian
2,Pomeranian,Pomeranian
3,Pomeranian,Pomeranian
4,Pomeranian,Pomeranian
...,...,...
2104,papillon,papillon
2105,papillon,papillon
2106,papillon,papillon
2107,papillon,papillon


## Confusion Matrix

In [10]:
# Print the confusion matrix
print(metrics.confusion_matrix(y_true, y_pred))


[[24  0  0 ...  0  0  0]
 [ 0 17  0 ...  0  0  0]
 [ 0  0 20 ...  0  0  0]
 ...
 [ 0  0  0 ... 15  0  0]
 [ 0  0  0 ...  0 14  0]
 [ 0  0  0 ...  0  0 15]]


## Precision, Recall and F1

In [11]:
# Print the precision and recall, among other metrics
print(metrics.classification_report(y_true, y_pred, digits=3))

                                precision    recall  f1-score   support

                  Afghan hound      0.828     1.000     0.906        24
           African hunting dog      0.895     1.000     0.944        17
                      Airedale      0.714     0.952     0.816        21
American Staffordshire terrier      0.857     0.353     0.500        17
                   Appenzeller      0.778     0.438     0.560        16
            Australian terrier      0.800     0.600     0.686        20
            Bedlington terrier      0.947     0.947     0.947        19
          Bernese mountain dog      0.808     0.955     0.875        22
              Blenheim spaniel      0.842     0.842     0.842        19
                 Border collie      0.688     0.733     0.710        15
                Border terrier      0.783     1.000     0.878        18
                   Boston bull      0.864     1.000     0.927        19
          Bouvier des Flandres      0.867     0.867     0.867  

## Macro F1 Score

In [12]:
from sklearn.metrics import f1_score

f1_score(y_true, y_pred, average='macro')


0.8190851377481553

## Micro F1 Score

In [13]:
f1_score(y_true, y_pred, average='micro')


0.8259838786154575

## Weighted F1 Score

In [14]:
f1_score(y_true, y_pred, average='weighted')

0.8215095090842068

## Cohen Kappa Score

In [15]:
from sklearn.metrics import cohen_kappa_score

cohen_kappa_score(y_true, y_pred)

0.8244872849013845

## Total Accuracy calculated from per class accuracy

In [16]:
# calculated from per class accuracy
total_accuracy = sum(per_class_accuracy.values())/120
total_accuracy

82.18854610301977

## Per Breed Accuracy

### Top 10 Breeds with least accuracy

In [17]:
k = 10
sorted(per_class_accuracy.items(), key=lambda x: x[1])[:k]

[('Eskimo dog', 13.333333333333334),
 ('American Staffordshire terrier', 35.294117647058826),
 ('Walker hound', 37.5),
 ('Appenzeller', 43.75),
 ('miniature poodle', 50.0),
 ('toy terrier', 50.0),
 ('Lhasa', 52.63157894736842),
 ('Irish terrier', 58.82352941176471),
 ('Australian terrier', 60.0),
 ('giant schnauzer', 62.5)]

### Top 10 Breeds with most accuracy

In [18]:
k = 10
sorted(per_class_accuracy.items(), key=lambda x: x[1], reverse=True)[:k]

[('Pomeranian', 100.0),
 ('Irish setter', 100.0),
 ('chow', 100.0),
 ('komondor', 100.0),
 ('keeshond', 100.0),
 ('Samoyed', 100.0),
 ('Rottweiler', 100.0),
 ('Boston bull', 100.0),
 ('Doberman', 100.0),
 ('Kerry blue terrier', 100.0)]